In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta


In [3]:

tgt_url = "https://giris.epias.com.tr/cas/v1/tickets"

payload = {
    "username": "", 
    "password": ""                       
}

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "text/plain"  
}

response = requests.post(tgt_url, data=payload, headers=headers)

if response.status_code == 201: 
    tgt = response.text.strip()
    print(f"TGT başarıyla alındı: {tgt}")
else:
    print(f"TGT alınırken hata oluştu! HTTP Kodu: {response.status_code}")
    print(response.text)


ConnectionError: HTTPSConnectionPool(host='giris.epias.com.tr', port=443): Max retries exceeded with url: /cas/v1/tickets (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x13c32e280>: Failed to resolve 'giris.epias.com.tr' ([Errno 8] nodename nor servname provided, or not known)"))

In [40]:
realtime_generation_url = "https://seffaflik.epias.com.tr/electricity-service/v1/generation/data/realtime-generation"

headers = {
    "Accept-Language": "en",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "TGT": "TGT-7240610-Uzjjk3WTGelX9XtMDBA2qqW3KUNXffMuyLdH9d8hzwD7-0-p1XNFrIm3STVCjErMXb0-cas-7bc549fccd-9hbsh"
}

start_date = datetime(2018, 12, 1)  
end_date = datetime(2024, 12, 1)    
step = timedelta(days=60)         

all_data = pd.DataFrame()

current_start = start_date
while current_start < end_date:
    current_end = current_start + step
    if current_end > end_date:
        current_end = end_date  
    
    body = {
        "startDate": current_start.strftime("%Y-%m-%dT%H:%M:%S+03:00"),
        "endDate": current_end.strftime("%Y-%m-%dT%H:%M:%S+03:00")
    }

    response = requests.post(realtime_generation_url, headers=headers, json=body)
    
    if response.status_code == 200:
        print(f"Veri çekildi: {current_start.date()} - {current_end.date()}")
        data = response.json()
        
        df = pd.DataFrame(data['items'])
        
        df['Tarih'] = pd.to_datetime(df['date']).dt.strftime('%d.%m.%Y')
        df.drop(columns=['date'], inplace=True)
        
        all_data = pd.concat([all_data, df], ignore_index=True)
    else:
        print(f"Veri çekilemedi: {current_start.date()} - {current_end.date()}, Hata: {response.text}")
    
    current_start = current_end

csv_file = "realtime_generation_data.csv"
all_data.to_csv(csv_file, index=False, encoding='utf-8-sig')
print(f"Tüm veriler {csv_file} dosyasına kaydedildi.")

Veri çekildi: 2018-12-01 - 2019-01-30
Veri çekildi: 2019-01-30 - 2019-03-31
Veri çekildi: 2019-03-31 - 2019-05-30
Veri çekildi: 2019-05-30 - 2019-07-29
Veri çekildi: 2019-07-29 - 2019-09-27
Veri çekildi: 2019-09-27 - 2019-11-26
Veri çekildi: 2019-11-26 - 2020-01-25
Veri çekildi: 2020-01-25 - 2020-03-25
Veri çekildi: 2020-03-25 - 2020-05-24
Veri çekildi: 2020-05-24 - 2020-07-23
Veri çekildi: 2020-07-23 - 2020-09-21
Veri çekildi: 2020-09-21 - 2020-11-20
Veri çekildi: 2020-11-20 - 2021-01-19
Veri çekildi: 2021-01-19 - 2021-03-20
Veri çekildi: 2021-03-20 - 2021-05-19
Veri çekildi: 2021-05-19 - 2021-07-18
Veri çekildi: 2021-07-18 - 2021-09-16
Veri çekildi: 2021-09-16 - 2021-11-15
Veri çekildi: 2021-11-15 - 2022-01-14
Veri çekildi: 2022-01-14 - 2022-03-15
Veri çekildi: 2022-03-15 - 2022-05-14
Veri çekildi: 2022-05-14 - 2022-07-13
Veri çekildi: 2022-07-13 - 2022-09-11
Veri çekildi: 2022-09-11 - 2022-11-10
Veri çekildi: 2022-11-10 - 2023-01-09
Veri çekildi: 2023-01-09 - 2023-03-10
Veri çekildi

In [41]:
duplicates = all_data[all_data.duplicated(subset=['Tarih', 'hour'], keep=False)]

if not duplicates.empty:
    print(f"Toplam çakışan kayıt sayısı: {duplicates.shape[0]}")
    print("Çakışan kayıtlar:")
    print(duplicates.sort_values(by=['Tarih', 'hour']))
else:
    print("Tarih ve saatlerde çakışma yok.")


Toplam çakışan kayıt sayısı: 1728
Çakışan kayıtlar:
        hour     total  naturalGas  dammedHydro  lignite    river  importCoal  \
49752  00:00  41899.84    10524.24      7399.78  5036.86  1829.25     9281.02   
49776  00:00  41899.84    10524.24      7399.78  5036.86  1829.25     9281.02   
49753  01:00  40098.43    10690.64      5825.20  5061.39  1644.83     9315.38   
49777  01:00  40098.43    10690.64      5825.20  5061.39  1644.83     9315.38   
49754  02:00  38551.95    10670.43      4546.33  5059.96  1552.45     9325.52   
...      ...       ...         ...          ...      ...      ...         ...   
51261  21:00  41920.25    11757.66      6996.70  5083.43  1395.79    10050.31   
51238  22:00  40243.11    11037.05      6501.74  5053.25  1218.64    10033.96   
51262  22:00  40243.11    11037.05      6501.74  5053.25  1218.64    10033.96   
51239  23:00  38850.57    10436.83      6094.28  5058.04  1228.16    10053.43   
51263  23:00  38850.57    10436.83      6094.28  5058.04 

In [42]:
duplicates = all_data[all_data.duplicated(subset=['Tarih', 'hour'], keep=False)]
if not duplicates.empty:
    print(f"Çakışan kayıt sayısı: {duplicates.shape[0]}")
else:
    print("Çakışan kayıt yok.")

unique_data = all_data.drop_duplicates(subset=['Tarih', 'hour'], keep='first')

unique_file = "realtime_generation_data.csv"
unique_data.to_csv(unique_file, index=False, encoding='utf-8-sig')

print(f"Benzersiz veriler {unique_file} dosyasına kaydedildi.")



Çakışan kayıt sayısı: 1728
Benzersiz veriler realtime_generation_data.csv dosyasına kaydedildi.


In [43]:
import pandas as pd

generation_file = "realtime_generation_data.csv"  
generation_data = pd.read_csv(generation_file)

null_counts = generation_data.isnull().sum()

null_columns = null_counts[null_counts > 0]
if not null_columns.empty:
    print("Bu sütunlarda null değerler var:")
    print(null_columns)
else:
    print("Veriler arasında null değer bulunmuyor")
    print(null_counts)




Veriler arasında null değer bulunmuyor
hour              0
total             0
naturalGas        0
dammedHydro       0
lignite           0
river             0
importCoal        0
wind              0
sun               0
fueloil           0
geothermal        0
asphaltiteCoal    0
blackCoal         0
biomass           0
naphta            0
lng               0
importExport      0
wasteheat         0
Tarih             0
dtype: int64


In [44]:
negative_values = generation_data.select_dtypes(include=['number']).lt(0).sum()

if negative_values.sum() > 0:
    print("Aşağıdaki sütunlarda negatif değerler bulunuyor:")
    print(negative_values[negative_values > 0])
else:
    print("Negatif değer bulunmuyor.")


Aşağıdaki sütunlarda negatif değerler bulunuyor:
sun                21
importExport    25206
dtype: int64


In [45]:
import pandas as pd

file_path = "realtime_generation_data.csv"  
generation_data = pd.read_csv(file_path)

columns_to_sum = generation_data.columns.difference(['Tarih', 'hour', 'total'])

generation_data['calculated_total'] = generation_data[columns_to_sum].sum(axis=1)
discrepancies = generation_data[generation_data['total'] != generation_data['calculated_total']]

large_discrepancies = discrepancies[abs(discrepancies['total'] - discrepancies['calculated_total']) > 0.00001]

if not large_discrepancies.empty:
    print(f"Fark olan satır sayısı: {large_discrepancies.shape[0]}")
else:
    print("Fark olan satır yok.")



Fark olan satır sayısı: 33


In [46]:
import pandas as pd

file_path = "realtime_generation_data.csv"  
generation_data = pd.read_csv(file_path)

columns_to_sum = generation_data.columns.difference(['Tarih', 'hour', 'total'])

generation_data['calculated_total'] = generation_data[columns_to_sum].sum(axis=1)

large_discrepancies = generation_data[abs(generation_data['total'] - generation_data['calculated_total']) > 0.00001]

if not large_discrepancies.empty:
    generation_data.loc[
        abs(generation_data['total'] - generation_data['calculated_total']) > 0.00001, 
        'total'
    ] = generation_data['calculated_total']
    print(f"'total' sütunu, fark bulunan {large_discrepancies.shape[0]} satır için düzeltildi.")
else:
    print("Düzeltilecek fark bulunamadı.")

generation_data.drop(columns=['calculated_total'], inplace=True)

updated_file_path = "realtime_generation_data.csv"
generation_data.to_csv(updated_file_path, index=False, encoding='utf-8-sig')

print(f"Düzeltilmiş dosya {updated_file_path} olarak kaydedildi.")


'total' sütunu, fark bulunan 33 satır için düzeltildi.
Düzeltilmiş dosya realtime_generation_data.csv olarak kaydedildi.


In [47]:
import pandas as pd

file_path = "realtime_generation_data.csv" 
generation_data = pd.read_csv(file_path)

total_generation_data = generation_data[['Tarih', 'hour', 'total']]

total_only_file = "realtime_generation_data.csv"
total_generation_data.to_csv(total_only_file, index=False, encoding='utf-8-sig')

print(f"Sadece 'total' sütunu kalan veri seti {total_only_file} dosyasına kaydedildi.")


Sadece 'total' sütunu kalan veri seti realtime_generation_data.csv dosyasına kaydedildi.


In [49]:
import requests
import pandas as pd
from datetime import datetime, timedelta

consumption_url = "https://seffaflik.epias.com.tr/electricity-service/v1/consumption/data/realtime-consumption"

headers = {
    "Accept-Language": "en",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "TGT": ""
}

start_date = datetime(2018, 12, 1)  
end_date = datetime(2024, 12, 1)    
step = timedelta(days=60)         

all_data = pd.DataFrame()

current_start = start_date
while current_start < end_date:
    current_end = current_start + step
    if current_end > end_date:
        current_end = end_date

    body = {
        "startDate": current_start.strftime("%Y-%m-%dT%H:%M:%S+03:00"),
        "endDate": current_end.strftime("%Y-%m-%dT%H:%M:%S+03:00")
    }

    response = requests.post(consumption_url, headers=headers, json=body)

    if response.status_code == 200:
        print(f"Veri çekildi: {current_start.date()} - {current_end.date()}")
        data = response.json()

        df = pd.DataFrame(data['items'])

        df['Tarih'] = pd.to_datetime(df['date']).dt.strftime('%d.%m.%Y')
        df['hour'] = pd.to_datetime(df['date']).dt.strftime('%H:%M')
        df.drop(columns=['date', 'time'], errors='ignore', inplace=True)  

        all_data = pd.concat([all_data, df], ignore_index=True)
    else:
        print(f"Veri çekilemedi: {current_start.date()} - {current_end.date()}, Hata: {response.text}")

    current_start = current_end

csv_file = "realtime_consumption_data.csv"
all_data.to_csv(csv_file, index=False, encoding='utf-8-sig')
print(f"Temizlenmiş tüketim verileri {csv_file} dosyasına kaydedildi.")


Veri çekildi: 2018-12-01 - 2019-01-30
Veri çekildi: 2019-01-30 - 2019-03-31
Veri çekildi: 2019-03-31 - 2019-05-30
Veri çekildi: 2019-05-30 - 2019-07-29
Veri çekildi: 2019-07-29 - 2019-09-27
Veri çekildi: 2019-09-27 - 2019-11-26
Veri çekildi: 2019-11-26 - 2020-01-25
Veri çekildi: 2020-01-25 - 2020-03-25
Veri çekildi: 2020-03-25 - 2020-05-24
Veri çekildi: 2020-05-24 - 2020-07-23
Veri çekildi: 2020-07-23 - 2020-09-21
Veri çekildi: 2020-09-21 - 2020-11-20
Veri çekildi: 2020-11-20 - 2021-01-19
Veri çekildi: 2021-01-19 - 2021-03-20
Veri çekildi: 2021-03-20 - 2021-05-19
Veri çekildi: 2021-05-19 - 2021-07-18
Veri çekildi: 2021-07-18 - 2021-09-16
Veri çekildi: 2021-09-16 - 2021-11-15
Veri çekildi: 2021-11-15 - 2022-01-14
Veri çekildi: 2022-01-14 - 2022-03-15
Veri çekildi: 2022-03-15 - 2022-05-14
Veri çekildi: 2022-05-14 - 2022-07-13
Veri çekildi: 2022-07-13 - 2022-09-11
Veri çekildi: 2022-09-11 - 2022-11-10
Veri çekildi: 2022-11-10 - 2023-01-09
Veri çekildi: 2023-01-09 - 2023-03-10
Veri çekildi

In [53]:
duplicates = all_data[all_data.duplicated(subset=['Tarih', 'hour'], keep=False)]

if not duplicates.empty:
    print(f"Toplam çakışan kayıt sayısı: {duplicates.shape[0]}")
    print("Çakışan kayıtlar:")
    print(duplicates.sort_values(by=['Tarih', 'hour']))
else:
    print("Tarih ve saatlerde çakışma yok.")



Toplam çakışan kayıt sayısı: 1728
Çakışan kayıtlar:
       consumption       Tarih   hour
49752     41929.91  02.07.2024  00:00
49776     41929.91  02.07.2024  00:00
49753     40124.03  02.07.2024  01:00
49777     40124.03  02.07.2024  01:00
49754     38578.77  02.07.2024  02:00
...            ...         ...    ...
51261     41920.25  31.08.2024  21:00
51238     40243.11  31.08.2024  22:00
51262     40243.11  31.08.2024  22:00
51239     38801.92  31.08.2024  23:00
51263     38801.92  31.08.2024  23:00

[1728 rows x 3 columns]


In [54]:
duplicates = all_data[all_data.duplicated(subset=['Tarih', 'hour'], keep=False)]
if not duplicates.empty:
    print(f"Çakışan kayıt sayısı: {duplicates.shape[0]}")
else:
    print("Çakışan kayıt yok.")

unique_data = all_data.drop_duplicates(subset=['Tarih', 'hour'], keep='first')

unique_file = "realtime_consumption_data.csv"
unique_data.to_csv(unique_file, index=False, encoding='utf-8-sig')

print(f"Benzersiz veriler {unique_file} dosyasına kaydedildi.")


Çakışan kayıt sayısı: 1728
Benzersiz veriler realtime_consumption_data.csv dosyasına kaydedildi.


In [55]:
import pandas as pd

generation_file = "realtime_consumption_data.csv"  
generation_data = pd.read_csv(generation_file)

null_counts = generation_data.isnull().sum()

null_columns = null_counts[null_counts > 0]
if not null_columns.empty:
    print("Bu sütunlarda null değerler var:")
    print(null_columns)
else:
    print("Veriler arasında null değer bulunmuyor")
    print(null_counts)





Veriler arasında null değer bulunmuyor
consumption    0
Tarih          0
hour           0
dtype: int64
